In [6]:
from pyspark.sql.functions import udf, col,split
from pyspark.ml.clustering import KMeans
import json

In [7]:
from pyspark.sql import SparkSession
my_spark = SparkSession \
    .builder \
    .appName("myApp") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/mydatabase.tweets_test") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/mydatabase.tweets_test") \
    .config("spark.io.compression.codec", "snappy").getOrCreate() #this line's config is for solving lz4 error
dataFrame=my_spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
dataFrame.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- loc_lat: double (nullable = true)
 |-- loc_long: double (nullable = true)
 |-- text: string (nullable = true)



In [8]:
#split = udf(lambda x: x.split(','))
#df.withColumn("user_location", split_udf(col("user_location"))).show()
#df=dataFrame.withColumn("user_location",
   # split(col("user_location"), ",\s*").cast("array<float>").alias("user_location")
#)
#df_loc = df.select('user_location')

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=['loc_lat','loc_long'],
    outputCol='features')
trainingData = assembler.transform(dataFrame)
#trainingData.show()

+--------------------+-------------------+-----------+-----------------+--------------------+--------------------+
|                 _id|         created_at|    loc_lat|         loc_long|                text|            features|
+--------------------+-------------------+-----------+-----------------+--------------------+--------------------+
|[5c9b0359cb60753c...|2019-03-27 04:58:53| 40.7308619|      -73.9871558|RT @TimInHonolulu...|[40.7308619,-73.9...|
|[5c9b0359cb60753c...|2019-03-27 04:58:53|  30.421309|      -87.2169149|RT @_alikingg: I ...|[30.421309,-87.21...|
|[5c9b0359cb60753c...|2019-03-27 04:58:53| 33.9533546|     -117.3961623|I need gfs that l...|[33.9533546,-117....|
|[5c9b0359cb60753c...|2019-03-27 04:58:53|  2.3923759|      112.8471939|RT @maddoooog: pl...|[2.3923759,112.84...|
|[5c9b0359cb60753c...|2019-03-27 04:58:53|40.85048545|-73.8404035580209|RT @crushcuIture:...|[40.85048545,-73....|
|[5c9b0359cb60753c...|2019-03-27 04:58:53|  1.1073992|      104.0803448|@MysticD

In [10]:
kmeans = KMeans(k=3,seed=1)
model = kmeans.fit(trainingData)
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[60.0977304 23.9197808]
[  37.15910751 -106.55457766]
[ 5.49296497 98.17294613]


In [19]:
cluster_ind = model.transform(trainingData)
dataToKibana=cluster_ind.toJSON().collect()

In [20]:
dataToKibana

['{"_id":{"oid":"5c9b0359cb60753cd01f6593"},"created_at":"2019-03-27 04:58:53","loc_lat":40.7308619,"loc_long":-73.9871558,"text":"RT @TimInHonolulu: 1. Barr\'s usurpation of the Special Counsel authority, having already acted to give free legal advice to the President o…","features":{"type":1,"values":[40.7308619,-73.9871558]},"prediction":1}',
 '{"_id":{"oid":"5c9b0359cb60753cd01f6594"},"created_at":"2019-03-27 04:58:53","loc_lat":30.421309,"loc_long":-87.2169149,"text":"RT @_alikingg: I just need all my girls to know that this one time I thought I was getting the flu...and it turned out to be a human. Stay…","features":{"type":1,"values":[30.421309,-87.2169149]},"prediction":1}',
 '{"_id":{"oid":"5c9b0359cb60753cd01f6595"},"created_at":"2019-03-27 04:58:53","loc_lat":33.9533546,"loc_long":-117.3961623,"text":"I need gfs that like dressing cute and taking pics for no reason","features":{"type":1,"values":[33.9533546,-117.3961623]},"prediction":1}',
 '{"_id":{"oid":"5c9b0359cb60753cd0